In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

In [3]:
import spacy
from re import sub, split


# Clean the data

In [4]:
ratings_raw = pd.read_csv('data/docs_before.csv')

In [5]:
ratings_raw.head()

,Unnamed: 0,hp_id,ratemds_id,order_id,hasorder,review_count,avg_help,avg_know,avg_punct,avg_staff,...,cat_nonprej,cat_otherprej,cat_restrictlicense,cat_losslicense,cem_strata_alldocs,basecat_badmed,basecat_impaired,basecat_finances,basecat_criminal,basecat_patient
0,1,303282,24616,NaN,0,3,4.333333,4.333333,4.333333,2.666667,...,NaN,NaN,NaN,NaN,7040,0,0,0,0,0
1,2,427422,66138,NaN,0,3,4.666667,5.000000,4.666667,3.333333,...,NaN,NaN,NaN,NaN,3843,0,0,0,0,0
2,3,19713,127113,NaN,0,11,2.272727,2.636364,3.000000,2.000000,...,NaN,NaN,NaN,NaN,2310,0,0,0,0,0
3,4,1250536,3065893,NaN,0,1,5.000000,5.000000,5.000000,5.000000,...,NaN,NaN,NaN,NaN,6945,0,0,0,0,0
4,5,1269847,284377,NaN,0,1,1.000000,2.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,2072,0,0,0,0,0


In [6]:
ratings_raw["avg_ranking"] = ratings_raw.iloc[:,6:10].mean(axis=1)


In [7]:
ratings_raw.columns

Index(['Unnamed: 0', 'hp_id', 'ratemds_id', 'order_id', 'hasorder',
       'review_count', 'avg_help', 'avg_know', 'avg_punct', 'avg_staff',
       'var_help', 'var_know', 'var_punct', 'var_staff', 'rem_neg_DONOTUSE',
       'rem_pos_DONOTUSE', 'ones', 'twos', 'threes', 'fours', 'fives',
       'prop_ones', 'prop_twos', 'prop_threes', 'prop_fours', 'prop_fives',
       'review_corpus', 'spec_comb', 'gy_comb', 'HP_ST', 'firstorderdate',
       'action_desc_concat', 'action_comment_concat', 'orderbasesconcat',
       'cat_nonprej', 'cat_otherprej', 'cat_restrictlicense',
       'cat_losslicense', 'cem_strata_alldocs', 'basecat_badmed',
       'basecat_impaired', 'basecat_finances', 'basecat_criminal',
       'basecat_patient', 'avg_ranking'],
      dtype='object')

In [8]:
useful_cols = ['hp_id',
               'ratemds_id',
               'hasorder',
               'order_id',
               'avg_help', 
               'avg_know', 
               'avg_punct', 
               'avg_staff',
               'avg_ranking',
               'spec_comb',
               'review_corpus'
              ]

In [9]:
ratings = ratings_raw[useful_cols]

In [10]:
ratings = ratings.drop_duplicates(subset='hp_id',keep='first')

In [11]:
ratings.head()

,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,spec_comb,review_corpus
0,303282,24616,0,NaN,4.333333,4.333333,4.333333,2.666667,3.916667,Family Practice,"Very clean offices, friendly and informative s..."
1,427422,66138,0,NaN,4.666667,5.000000,4.666667,3.333333,4.416667,Family Practice,If you are lucky enough to get in to see Dr. B...
2,19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,Internal Medicine,"satisfactory but would not recommend coldness,..."
3,1250536,3065893,0,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,Family Practice,The best physician I've been to in my 70 years...
4,1269847,284377,0,NaN,1.000000,2.000000,1.000000,1.000000,1.250000,Surgery,I found Dr. Bernstein to be cold and uncaring....


In [12]:
ratings

,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,spec_comb,review_corpus
0,303282,24616,0,NaN,4.333333,4.333333,4.333333,2.666667,3.916667,Family Practice,"Very clean offices, friendly and informative s..."
1,427422,66138,0,NaN,4.666667,5.000000,4.666667,3.333333,4.416667,Family Practice,If you are lucky enough to get in to see Dr. B...
2,19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,Internal Medicine,"satisfactory but would not recommend coldness,..."
3,1250536,3065893,0,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,Family Practice,The best physician I've been to in my 70 years...
4,1269847,284377,0,NaN,1.000000,2.000000,1.000000,1.000000,1.250000,Surgery,I found Dr. Bernstein to be cold and uncaring....
...,...,...,...,...,...,...,...,...,...,...,...
134968,1165896,2474959,0,NaN,2.000000,1.000000,2.000000,2.000000,1.750000,Family Practice,Very nice but not genuine-seeming. Didn't have...
134969,1049044,3205133,0,NaN,2.666667,2.666667,3.000000,3.000000,2.833333,Internal Medicine,"dr. Foster has been excellent, she has hapled ..."
134970,1132315,2880773,0,NaN,3.666667,4.333333,3.333333,4.333333,3.916667,Internal Medicine,Dr. Davis is a very caring physician. She is n...
134971,1308120,152018,0,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,Pediatrics,"Two years ago, I was diagnosed with ulcerative..."


In [13]:
gender_info = pd.read_csv('data/genders.csv')

In [14]:
gender_info.drop_duplicates(subset='hp_id', keep='first',inplace = True)

# Merge Datasets

In [15]:
doc_review = pd.merge(ratings, gender_info, on='hp_id', how='left')

In [16]:
doc_review['spec_comb'] = doc_review['spec_comb'].map(lambda x: x.strip())

In [17]:
doc_review['sentence']= doc_review['review_corpus'].map(lambda x: x.split('|'))

In [18]:
for sentence in doc_review['sentence'][0]:
    print(sentence)

Very clean offices, friendly and informative staff 
 I love this doctor! She is very warm and takes her time to answer questions. She's in an office with many doctors, and I am usually glad when she's the doctor on call. 
 She's really nice and helpful. She takes time to listen to what's going on in order to come up with the best solution. I really like her! 


In [19]:
len(doc_review)

134968

In [20]:
len(doc_review['hp_id'].unique())

134968

In [21]:
doc_review

,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,spec_comb,review_corpus,gender,sentence
0,303282,24616,0,NaN,4.333333,4.333333,4.333333,2.666667,3.916667,Family Practice,"Very clean offices, friendly and informative s...",F,"[Very clean offices, friendly and informative ..."
1,427422,66138,0,NaN,4.666667,5.000000,4.666667,3.333333,4.416667,Family Practice,If you are lucky enough to get in to see Dr. B...,M,[If you are lucky enough to get in to see Dr. ...
2,19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,Internal Medicine,"satisfactory but would not recommend coldness,...",F,[satisfactory but would not recommend coldness...
3,1250536,3065893,0,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,Family Practice,The best physician I've been to in my 70 years...,F,[The best physician I've been to in my 70 year...
4,1269847,284377,0,NaN,1.000000,2.000000,1.000000,1.000000,1.250000,Surgery,I found Dr. Bernstein to be cold and uncaring....,F,[I found Dr. Bernstein to be cold and uncaring...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134963,1165896,2474959,0,NaN,2.000000,1.000000,2.000000,2.000000,1.750000,Family Practice,Very nice but not genuine-seeming. Didn't have...,F,[Very nice but not genuine-seeming. Didn't hav...
134964,1049044,3205133,0,NaN,2.666667,2.666667,3.000000,3.000000,2.833333,Internal Medicine,"dr. Foster has been excellent, she has hapled ...",F,"[dr. Foster has been excellent, she has hapled..."
134965,1132315,2880773,0,NaN,3.666667,4.333333,3.333333,4.333333,3.916667,Internal Medicine,Dr. Davis is a very caring physician. She is n...,F,[Dr. Davis is a very caring physician. She is ...
134966,1308120,152018,0,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,Pediatrics,"Two years ago, I was diagnosed with ulcerative...",M,"[Two years ago, I was diagnosed with ulcerativ..."


# Create a datset where there is one row for review not for doctor

In [22]:
doc_review.reset_index(inplace= True,drop = True)

In [23]:
doc_review.head()

,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,spec_comb,review_corpus,gender,sentence
0,303282,24616,0,NaN,4.333333,4.333333,4.333333,2.666667,3.916667,Family Practice,"Very clean offices, friendly and informative s...",F,"[Very clean offices, friendly and informative ..."
1,427422,66138,0,NaN,4.666667,5.000000,4.666667,3.333333,4.416667,Family Practice,If you are lucky enough to get in to see Dr. B...,M,[If you are lucky enough to get in to see Dr. ...
2,19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,Internal Medicine,"satisfactory but would not recommend coldness,...",F,[satisfactory but would not recommend coldness...
3,1250536,3065893,0,NaN,5.000000,5.000000,5.000000,5.000000,5.000000,Family Practice,The best physician I've been to in my 70 years...,F,[The best physician I've been to in my 70 year...
4,1269847,284377,0,NaN,1.000000,2.000000,1.000000,1.000000,1.250000,Surgery,I found Dr. Bernstein to be cold and uncaring....,F,[I found Dr. Bernstein to be cold and uncaring...


In [24]:
im_reviews = doc_review[doc_review["spec_comb"] == "Internal Medicine"].reset_index(drop = True)

In [25]:
im_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33549 entries, 0 to 33548
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hp_id          33549 non-null  int64  
 1   ratemds_id     33549 non-null  int64  
 2   hasorder       33549 non-null  int64  
 3   order_id       207 non-null    float64
 4   avg_help       33549 non-null  float64
 5   avg_know       33549 non-null  float64
 6   avg_punct      33549 non-null  float64
 7   avg_staff      31987 non-null  float64
 8   avg_ranking    33549 non-null  float64
 9   spec_comb      33549 non-null  object 
 10  review_corpus  33549 non-null  object 
 11  gender         33549 non-null  object 
 12  sentence       33549 non-null  object 
dtypes: float64(6), int64(3), object(4)
memory usage: 3.3+ MB


In [26]:
# Generate a new dataframe of doc's review, make sure each record has single review
im_review_df = pd.DataFrame()
for i, sentence in enumerate(im_reviews['sentence']):
    temp_dict = dict(enumerate(sentence))
    a = len(list(temp_dict.keys()))
    s = str(im_reviews['hp_id'][i])
    temp_df = pd.DataFrame.from_dict(data = temp_dict, orient = 'index', columns=['Review'])
    temp_df['hp_id'] = [s for i in range(a)]
    im_review_df = im_review_df.append(temp_df, ignore_index=True)

In [27]:
im_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89661 entries, 0 to 89660
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  89661 non-null  object
 1   hp_id   89661 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [28]:
im_review_df

,Review,hp_id
0,"satisfactory but would not recommend coldness,...",19713
1,Although Dr. Merlo does not appear to have an...,19713
2,"took my daughter for problems, never examined...",19713
3,"Yes, Dr. Merlot can be rude and arrogant, but...",19713
4,I called to make and appointment and a woman ...,19713
...,...,...
89656,"This doctor is just flaky. Dr. Foster, just s...",1049044
89657,DO NOT GO TO DR. ERNIA FOSTER. She perforated...,1049044
89658,Dr. Davis is a very caring physician. She is n...,1132315
89659,I found it sad that Dr. Charlyce Davis will n...,1132315


In [29]:
im_review_df['hp_id'] = im_review_df['hp_id'].map(lambda x: int(x))

In [30]:
im_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89661 entries, 0 to 89660
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  89661 non-null  object
 1   hp_id   89661 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [31]:
im_review_df = pd.merge(im_review_df, doc_review, on='hp_id', how = 'left')

In [32]:
im_review_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89661 entries, 0 to 89660
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Review         89661 non-null  object 
 1   hp_id          89661 non-null  int64  
 2   ratemds_id     89661 non-null  int64  
 3   hasorder       89661 non-null  int64  
 4   order_id       593 non-null    float64
 5   avg_help       89661 non-null  float64
 6   avg_know       89661 non-null  float64
 7   avg_punct      89661 non-null  float64
 8   avg_staff      87664 non-null  float64
 9   avg_ranking    89661 non-null  float64
 10  spec_comb      89661 non-null  object 
 11  review_corpus  89661 non-null  object 
 12  gender         89661 non-null  object 
 13  sentence       89661 non-null  object 
dtypes: float64(6), int64(3), object(5)
memory usage: 10.3+ MB


In [33]:
im_review_df = im_review_df.drop(['review_corpus', 'sentence', 'spec_comb'], axis=1)


In [34]:
im_review_df.head()

,Review,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,gender
0,"satisfactory but would not recommend coldness,...",19713,127113,0,NaN,2.272727,2.636364,3.0,2.0,2.477273,F
1,Although Dr. Merlo does not appear to have an...,19713,127113,0,NaN,2.272727,2.636364,3.0,2.0,2.477273,F
2,"took my daughter for problems, never examined...",19713,127113,0,NaN,2.272727,2.636364,3.0,2.0,2.477273,F
3,"Yes, Dr. Merlot can be rude and arrogant, but...",19713,127113,0,NaN,2.272727,2.636364,3.0,2.0,2.477273,F
4,I called to make and appointment and a woman ...,19713,127113,0,NaN,2.272727,2.636364,3.0,2.0,2.477273,F


In [35]:
im_review_df.gender.value_counts()

M    63281
F    26380
Name: gender, dtype: int64

In [36]:
im_review_df.gender.value_counts()/len(im_review_df)

M    0.705781
F    0.294219
Name: gender, dtype: float64

In [37]:
im_review_df.hasorder.value_counts()

0    89068
1      593
Name: hasorder, dtype: int64

In [38]:
im_review_df.hasorder.value_counts()/len(im_review_df)

0    0.993386
1    0.006614
Name: hasorder, dtype: float64

In [39]:
def cols_to_binary(val, col_name):
    # col names can be "gender" or "ranking"
    if col_name == "gender":
        if val == "F":
            return_var = 1
        elif val == "M":
            return_var = 0
        else:
            return_var = "NA"
    elif col_name == "ranking":
        if val >= 4.0:
            return_var = 1
        elif val < 4.0:
            return_var = 0
        else: 
            return_var = "NA"
    return return_var

In [40]:
im_review_df["gen_bin"] = im_review_df["gender"].apply(lambda x: cols_to_binary(x, "gender"))
im_review_df["high_avg_rank"] = im_review_df["avg_ranking"].apply(lambda x: cols_to_binary(x, "ranking"))
im_review_df["high_avg_help"] = im_review_df["avg_help"].apply(lambda x: cols_to_binary(x, "ranking"))
im_review_df["high_avg_know"] = im_review_df["avg_know"].apply(lambda x: cols_to_binary(x, "ranking"))
im_review_df["high_avg_punc"] = im_review_df["avg_punct"].apply(lambda x: cols_to_binary(x, "ranking"))
im_review_df["high_avg_staf"] = im_review_df["avg_staff"].apply(lambda x: cols_to_binary(x, "ranking"))


In [41]:
im_review_df

,Review,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,gender,gen_bin,high_avg_rank,high_avg_help,high_avg_know,high_avg_punc,high_avg_staf
0,"satisfactory but would not recommend coldness,...",19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,F,1,0,0,0,0,0
1,Although Dr. Merlo does not appear to have an...,19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,F,1,0,0,0,0,0
2,"took my daughter for problems, never examined...",19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,F,1,0,0,0,0,0
3,"Yes, Dr. Merlot can be rude and arrogant, but...",19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,F,1,0,0,0,0,0
4,I called to make and appointment and a woman ...,19713,127113,0,NaN,2.272727,2.636364,3.000000,2.000000,2.477273,F,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89656,"This doctor is just flaky. Dr. Foster, just s...",1049044,3205133,0,NaN,2.666667,2.666667,3.000000,3.000000,2.833333,F,1,0,0,0,0,0
89657,DO NOT GO TO DR. ERNIA FOSTER. She perforated...,1049044,3205133,0,NaN,2.666667,2.666667,3.000000,3.000000,2.833333,F,1,0,0,0,0,0
89658,Dr. Davis is a very caring physician. She is n...,1132315,2880773,0,NaN,3.666667,4.333333,3.333333,4.333333,3.916667,F,1,0,0,1,0,1
89659,I found it sad that Dr. Charlyce Davis will n...,1132315,2880773,0,NaN,3.666667,4.333333,3.333333,4.333333,3.916667,F,1,0,0,1,0,1


# Now prepare the data for training

In [42]:
# shuffle them
df = im_review_df.sample(frac=1).reset_index(drop=True)


In [43]:
eighty_percent = len(df)*.8

71728.8

In [44]:
# take 80% for train // 20% for test
df_train = df[0:eighty_percent]
df_test = df[eighty_percent:]


In [45]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [46]:
df_train

,Review,hp_id,ratemds_id,hasorder,order_id,avg_help,avg_know,avg_punct,avg_staff,avg_ranking,gender,gen_bin,high_avg_rank,high_avg_help,high_avg_know,high_avg_punc,high_avg_staf
0,After almost 15 years of Dr's not being able ...,471809,115026,0,NaN,3.727273,3.363636,2.090909,3.636364,3.204545,M,0,0,0,0,0,0
1,I read the other comments and was so surprise...,1182870,802317,0,NaN,2.090909,2.636364,3.818182,3.545455,3.022727,M,0,0,0,0,0,0
2,With all due respect to the patient who said ...,414404,27184,0,NaN,2.800000,2.800000,4.200000,0.600000,2.600000,F,1,0,0,0,1,0
3,Dr Steiner is a very knowledgable and caring ...,569651,103627,0,NaN,5.000000,5.000000,5.000000,3.500000,4.625000,F,1,1,1,1,1,0
4,I went to him because my husbands family all g...,765142,715854,0,NaN,2.000000,1.500000,3.500000,3.500000,2.625000,M,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71724,Dr. Stone treats both me and my husband. We l...,307737,153207,0,NaN,4.666667,5.000000,5.000000,4.333333,4.750000,M,0,1,1,1,1,1
71725,Such an amazing doctor...used to be a Profess...,999182,27922,0,NaN,2.733333,3.000000,3.333333,2.000000,2.766667,M,0,0,0,0,0,0
71726,Dr. Fisher office staff is very unfriendly. It...,1294820,15780,0,NaN,3.000000,3.000000,2.000000,0.250000,2.062500,M,0,0,0,0,0,0
71727,Spends the time with me and does answer my que...,821968,2586678,0,NaN,4.000000,4.000000,3.000000,3.000000,3.500000,M,0,0,1,1,0,0


In [47]:
# split x data (literally just the review) from the other potential y variables (which also serve as metadata analysis features)

x_train = df_train["Review"]
y_train_gen = df_train["gen_bin"]
y_train_rank = df_train["high_avg_rank"]
y_train_help = df_train["high_avg_help"]
y_train_know = df_train["high_avg_know"]
y_train_punc = df_train["high_avg_punc"]

x_test = df_test["Review"]
y_test_gen = df_test["gen_bin"]
y_test_rank = df_test["high_avg_rank"]
y_test_help = df_test["high_avg_help"]
y_test_know = df_test["high_avg_know"]
y_test_punc = df_test["high_avg_punc"]

# Clean Explanations

In [48]:
x_train

0         After almost 15 years of Dr's not being able ...
1         I read the other comments and was so surprise...
2         With all due respect to the patient who said ...
3         Dr Steiner is a very knowledgable and caring ...
4        I went to him because my husbands family all g...
                               ...                        
71724     Dr. Stone treats both me and my husband. We l...
71725     Such an amazing doctor...used to be a Profess...
71726    Dr. Fisher office staff is very unfriendly. It...
71727    Spends the time with me and does answer my que...
71728    Excellent. knowledgeable, deliberate, responsi...
Name: Review, Length: 71729, dtype: object

In [49]:
def clean_word(s):
    
    # unwanted symbols/punctuations
    s = sub("%", " percent", s) ##percents
    s = sub("&amp;", "and", s) ##ampersands
    s = sub("'s", "", s) ##possessive or contraction
    s = sub("'re", "", s) ##contraction
    s = sub("'ll", "", s) ##contraction
    s = sub("‚Äô", "", s) ##contraction
    s = sub("'t", " not", s) ##contraction
    
    # typical text mining things
    s = s.lower() ##case sensitivity

    return s

In [50]:
x_train = x_train.apply(lambda x: clean_word(x))
x_test = x_test.apply(lambda x: clean_word(x))


## The most basic model: Naive Bayes

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorize text reviews to numbers
vec = CountVectorizer(stop_words='english')
x_train_vec = vec.fit_transform(x_train).toarray()
x_test_vec = vec.transform(x_test).toarray()


In [53]:
from sklearn.naive_bayes import MultinomialNB

model_nb = MultinomialNB()
model_nb.fit(x_train_vec, y_train_gen)

In [53]:
model_nb.score(x_train_vec, y_train_gen)


0.81757727000237

In [54]:
model_nb.score(x_test_vec, y_test_gen)


0.7440330136069596

In [55]:
# Predicting the Test set results
y_pred = model_nb.predict(x_test_vec)
 

In [54]:
def conf_matrix(pred_val, actual_val):
    if pred_val == 1:
        if actual_val == 1:
            return "TP"
        elif actual_val == 0:
            return "FP"
    elif pred_val == 0:
        if actual_val == 1:
            return "FN"
        elif actual_val == 0:
            return "TN"
    else:
        return "ERROR IN DATA"

In [55]:
def create_conf_df(model, x_df, y_df):
    compared_df = pd.DataFrame()
    compared_df["preds"] = model.predict(x_df)
    compared_df["actuals"] = y_df
    compared_df["error_type"] = compared_df.apply(lambda x: conf_matrix(x.preds, x.actuals), axis=1)
    compared_df["count"] = 1
    return compared_df

In [56]:
def summary_cost_matrix(model, x_df, y_df):
    conf_df = create_conf_df(model, x_df, y_df)
    summary_df = conf_df[["error_type", "count"]].groupby(by=["error_type"]).sum()
    fn = summary_df.iloc[0][0]
    fp = summary_df.iloc[1][0]
    tn = summary_df.iloc[2][0]
    tp = summary_df.iloc[3][0]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = (2*precision*recall)/(precision+recall)
    F0_5score = ((1 + 0.5**2) * precision * recall) / (0.5**2 * precision + recall)
    print(summary_df)
    print("FN:", fn)
    print("FP:", fp)
    print("TN:", tn)
    print("TP:", tp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1_score)
    print("F0.5:", F0_5score)

In [67]:
summary_cost_matrix(model_nb, x_train_vec, y_train_gen)

            count
error_type       
FN          10699
FP           2386
TN          48184
TP          10460
FN: 10699
FP: 2386
TN: 48184
TP: 10460
Precision: 0.8142612486377082
Recall: 0.49435228507963513
F1: 0.6152036465225702
F0.5: 0.7209517113987566


In [82]:
summary_cost_matrix(model_nb, x_test_vec, y_test_gen)

            count
error_type       
FN           3577
FP           1013
TN          11698
TP           1644
FN: 3577
FP: 1013
TN: 11698
TP: 1644
Precision: 0.6187429431689876
Recall: 0.31488220647385556
F1: 0.41736481340441733
F0.5: 0.5186447094453909


In [85]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
 
# n_estimators = # of trees
rf_model_no_extra = RandomForestClassifier(n_estimators = 501,
                                           criterion = 'entropy')
                             
rf_model_no_extra.fit(x_train_vec, y_train_gen)

In [ ]:
rf_model_no_extra.score(x_train_vec, y_train_gen)


In [ ]:
rf_model_no_extra.score(x_test_vec, y_test_gen)


In [ ]:
# Predicting the Test set results
y_pred_rf_no_extra = rf_model_no_extra.predict(x_test_vec)
 

In [ ]:
summary_cost_matrix(rf_model_no_extra, x_train_vec, y_train_gen)

In [ ]:
summary_cost_matrix(rf_model_no_extra, x_test_vec, y_test_gen)

# Adding lemmatization, stemming etc

In [57]:
# Tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

word_tokenize(x_train[0])

['after',
 'almost',
 '15',
 'years',
 'of',
 'dr',
 'not',
 'being',
 'able',
 'to',
 'find',
 'out',
 'what',
 'was',
 'wrong',
 'with',
 'my',
 'stomach',
 'issues',
 ',',
 'it',
 'took',
 'just',
 '2',
 'visits',
 'with',
 'dr.',
 'follick',
 'to',
 'find',
 'out',
 'it',
 'was',
 'crohn',
 'disease',
 'and',
 'malabsorption',
 '.',
 'now',
 'i',
 'feel',
 '1000x',
 'better',
 '.',
 'he',
 'has',
 'an',
 'amazing',
 'sense',
 'of',
 'humor',
 ',',
 'is',
 'extremely',
 'knowledgeable',
 'and',
 'has',
 'the',
 'tenacity',
 'of',
 'a',
 'pit',
 'bull',
 'when',
 'it',
 'comes',
 'to',
 'a',
 'diagnosis',
 '.',
 'sure',
 'some',
 'people',
 'may',
 'complain',
 'that',
 'the',
 'waiting',
 'room',
 'is',
 'crowded',
 'but',
 'would',
 'you',
 'feel',
 'better',
 'going',
 'to',
 'a',
 'dr.',
 'who',
 'has',
 'no',
 'one',
 'waiting',
 'to',
 'see',
 'him',
 '?']

In [58]:
# Lemmatize
lemm = WordNetLemmatizer()

def lemm_sentence(tokenized_sentence):
    new_tokenized_sentence = []
    for word in tokenized_sentence:
        new_tokenized_sentence.append(lemm.lemmatize(word))
    return new_tokenized_sentence

In [59]:
# x_train_lemmatized = x_train_tokenized.apply(lambda x: lemm_sentence(x))
# x_test_lemmatized = x_test_tokenized.apply(lambda x: lemm_sentence(x))

In [60]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
 
def stem_sentence(lemmatized_sentence):
    new_lemmatized_sentence = []
    for word in lemmatized_sentence:
        new_lemmatized_sentence.append(ps.stem(word))
    return new_lemmatized_sentence


In [61]:
# x_train_stemmed = x_train_lemmatized.apply(lambda x: stem_sentence(x))
# x_test_stemmed = x_test_lemmatized.apply(lambda x: stem_sentence(x))

In [62]:
def norm_sentence_again(stem_sentence):
    new_norm_sentence = ""
    for word in stem_sentence:
        new_norm_sentence = new_norm_sentence + " " + word
    return new_norm_sentence

In [63]:
# x_train_to_vec = x_train_stemmed.apply(lambda x: norm_sentence_again(x))
# x_test_to_vec = x_test_stemmed.apply(lambda x: norm_sentence_again(x))

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 100000)

In [65]:
def create_bigram_strings(string_sentence):
    bigram_list = []
    for word in range(0, len(string_sentence.split())-1):
        unigram_1 = string_sentence.split()[word]
        unigram_2 = string_sentence.split()[word+1]
        bigram_list.append(unigram_1 + "_" + unigram_2)
    bigram_sentence = ""
    for bigram in bigram_list:
        bigram_sentence = bigram_sentence + " " + bigram
    return bigram_sentence

In [66]:
def train_ratings_to_vectors(x_train, y_train, x_test, y_test):
    
    # clean them
    inner_x_train = x_train.apply(lambda x: clean_word(x))
    inner_x_test = x_test.apply(lambda x: clean_word(x))

    # tokenize
    inner_x_train_tokenized = inner_x_train.apply(lambda x: word_tokenize(x))
    inner_x_test_tokenized = inner_x_test.apply(lambda x: word_tokenize(x))

    # lemmatize
    lemm = WordNetLemmatizer()
    inner_x_train_lemmatized = inner_x_train_tokenized.apply(lambda x: lemm_sentence(x))
    inner_x_test_lemmatized = inner_x_test_tokenized.apply(lambda x: lemm_sentence(x))
    
    # stem
    ps = PorterStemmer()
    inner_x_train_stemmed = inner_x_train_lemmatized.apply(lambda x: stem_sentence(x))
    inner_x_test_stemmed = inner_x_test_lemmatized.apply(lambda x: stem_sentence(x))
    
    # return to non-vectorized-list so we can manipulate
    inner_x_train_to_vec = inner_x_train_stemmed.apply(lambda x: norm_sentence_again(x))
    inner_x_test_to_vec = inner_x_test_stemmed.apply(lambda x: norm_sentence_again(x))
    
    # now count vectorize
    cv = CountVectorizer()
    inner_x_train_vec = cv.fit_transform(pd.Series(inner_x_train_to_vec)).toarray()
    inner_x_test_vec = cv.transform(pd.Series(inner_x_test_to_vec)).toarray()

    return inner_x_train_vec, y_train, inner_x_test_vec, y_test

In [67]:
def train_ratings_to_vectors_bigram(x_train, y_train, x_test, y_test):
    # clean them
    inner_x_train = x_train.apply(lambda x: clean_word(x))
    inner_x_test = x_test.apply(lambda x: clean_word(x))

    # tokenize
    inner_x_train_tokenized = inner_x_train.apply(lambda x: word_tokenize(x))
    inner_x_test_tokenized = inner_x_test.apply(lambda x: word_tokenize(x))

    # lemmatize
    lemm = WordNetLemmatizer()
    inner_x_train_lemmatized = inner_x_train_tokenized.apply(lambda x: lemm_sentence(x))
    inner_x_test_lemmatized = inner_x_test_tokenized.apply(lambda x: lemm_sentence(x))
    
    # stem
    ps = PorterStemmer()
    inner_x_train_stemmed = inner_x_train_lemmatized.apply(lambda x: stem_sentence(x))
    inner_x_test_stemmed = inner_x_test_lemmatized.apply(lambda x: stem_sentence(x))
    
    # return to non-vectorized-list so we can manipulate
    inner_x_train_to_vec = inner_x_train_stemmed.apply(lambda x: norm_sentence_again(x))
    inner_x_test_to_vec = inner_x_test_stemmed.apply(lambda x: norm_sentence_again(x))
    
    # convert this guy to a bigrammized sentence
    bigrammed_sentence_x_train = inner_x_train_to_vec.apply(lambda x: create_bigram_strings(x))
    bigrammed_sentence_x_test  = inner_x_test_to_vec.apply(lambda x: create_bigram_strings(x))
    
    # make them one big sentence:
    comb_inner_x_train_to_vec = inner_x_train_to_vec + bigrammed_sentence_x_train
    comb_inner_x_test_to_vec = inner_x_test_to_vec + bigrammed_sentence_x_test
    
    # now count vectorize
    cv = CountVectorizer()
    inner_x_train_vec = cv.fit_transform(pd.Series(comb_inner_x_train_to_vec)).toarray()
    inner_x_test_vec = cv.transform(pd.Series(comb_inner_x_test_to_vec)).toarray()

    return inner_x_train_vec, y_train, inner_x_test_vec, y_test


In [68]:
bi_uni_df = train_ratings_to_vectors_bigram(x_train, y_train_gen, x_test, y_test_gen)

In [69]:
standard_df = train_ratings_to_vectors(x_train, y_train_gen, x_test, y_test_gen)

In [70]:
def all_eval_metrics(trained_model, x_train, y_train, x_test, y_test):
   
    # vectorized usable versions only of x train
    
    train_fit = trained_model.score(x_train, y_train)
    print("Train fit: ", train_fit)
    test_fit = trained_model.score(x_test, y_test)
    print("Test fit: ", test_fit)

    # Predicting the Test set results
    y_predictions = trained_model.predict(x_test)

    four_outputs = prec_reca_f1(y_test, y_predictions)
    print("Precision: ", four_outputs[0])
    print("Recall: ", four_outputs[1])
    print("F1: ", four_outputs[2])
    print("F0.5: ", four_outputs[3])


# Mix in the dictionaries

In [ ]:
# First we have to redo the tokenization because we want it to be in a different format to compute cosine similarity, we will not use these after we have the dictionary cosine similarity vals

In [71]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [72]:
# Lemmatize
lemm = WordNetLemmatizer()

def lemm_sentence(tokenized_sentence):
    new_tokenized_sentence = []
    for word in tokenized_sentence:
        new_tokenized_sentence.append(lemm.lemmatize(word))
    return new_tokenized_sentence

In [73]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
 
def stem_sentence(lemmatized_sentence):
    new_lemmatized_sentence = []
    for word in lemmatized_sentence:
        new_lemmatized_sentence.append(ps.stem(word))
    return new_lemmatized_sentence


In [74]:
def norm_sentence_again(stem_sentence):
    new_norm_sentence = ""
    for word in stem_sentence:
        new_norm_sentence = new_norm_sentence + " " + word
    return new_norm_sentence

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)

In [76]:
def train_ratings_to_vectors_STEMMED(x_train, y_train, x_test, y_test):
    # clean them
    inner_x_train = x_train.apply(lambda x: clean_word(x))
    inner_x_test = x_test.apply(lambda x: clean_word(x))

    # tokenize
    inner_x_train_tokenized = inner_x_train.apply(lambda x: word_tokenize(x))
    inner_x_test_tokenized = inner_x_test.apply(lambda x: word_tokenize(x))

    # lemmatize
    lemm = WordNetLemmatizer()
    inner_x_train_lemmatized = inner_x_train_tokenized.apply(lambda x: lemm_sentence(x))
    inner_x_test_lemmatized = inner_x_test_tokenized.apply(lambda x: lemm_sentence(x))
    
    # stem
    ps = PorterStemmer()
    inner_x_train_stemmed = inner_x_train_lemmatized.apply(lambda x: stem_sentence(x))
    inner_x_test_stemmed = inner_x_test_lemmatized.apply(lambda x: stem_sentence(x))
    
#     # return to non-vectorized-list so we can manipulate
#     inner_x_train_to_vec = inner_x_train_stemmed.apply(lambda x: norm_sentence_again(x))
#     inner_x_test_to_vec = inner_x_test_stemmed.apply(lambda x: norm_sentence_again(x))
    
#     # now count vectorize
#     cv = CountVectorizer(max_features = 1500)
#     inner_x_train_vec = cv.fit_transform(pd.Series(inner_x_train_to_vec)).toarray()
#     inner_x_test_vec = cv.transform(pd.Series(inner_x_test_to_vec)).toarray()

#    return inner_x_train_vec, inner_y_train, inner_x_test_vec, inner_y_test
    return inner_x_train_stemmed, y_train, inner_x_test_stemmed, y_test


In [77]:
standard_df_for_str = train_ratings_to_vectors_STEMMED(x_train, y_train_gen, x_test, y_test_gen)

In [78]:
def tokenized_to_string(tokenized_sent):
    new_sent = ""
    for word in tokenized_sent:
        new_sent = new_sent + " " + word
    return new_sent[1:]

In [79]:
standard_df_str = (standard_df_for_str[0].apply(tokenized_to_string),
                   standard_df_for_str[1],
                   standard_df_for_str[2].apply(tokenized_to_string),
                   standard_df_for_str[3],)

# Make covariate columns

In [80]:
mass_dict_pd = pd.read_excel('data/LIWC2007dictionary_cleaned.xls', sheet_name = "Cleaned")

In [81]:
mass_dict = {}

In [82]:
mass_dict_pd.columns

Index(['Quant', 'Numbers', 'Humans', 'Affect', 'Posemo', 'Negemo', 'Cause',
       'Health', 'Money', 'Death', 'Ipron', 'aux_verb', 'adverbs', 'Negate',
       'Family', 'Anx', 'Anger', 'Sad', 'CogMech', 'Insight', 'Discrep',
       'Tentat', 'Certain', 'Inhib', 'Incl', 'Excl', 'Bio', 'Body', 'Sexual',
       'Time', 'Achiev'],
      dtype='object')

In [83]:
for col in mass_dict_pd.columns:
    mass_dict[col]  = []
    for word in range(0, len(mass_dict_pd[col])):
        if pd.notna(mass_dict_pd[col][word]):
            clean_word = mass_dict_pd[col][word].replace("*", "")
            mass_dict[col].append(clean_word)

In [84]:
mass_dict

{'Quant': ['all',
  'allot',
  'alot',
  'amount',
  'another',
  'any',
  'anymore',
  'besides',
  'best ',
  'bit ',
  'bits',
  'both',
  'bunch',
  'cetera',
  'couple',
  'difference',
  'doubl',
  'each',
  'either',
  'else',
  'enough',
  'entire',
  'equal',
  'etc',
  'every',
  'example',
  'extent',
  'extra',
  'extremely',
  'fairly',
  'few',
  'form',
  'full',
  'greater',
  'greatest',
  'highly',
  'increas',
  'item',
  'lack',
  'least',
  'less',
  'loads',
  'lot',
  'lotof',
  'lots',
  'lotsa',
  'lotta',
  'main',
  'major',
  'majority',
  'many',
  'more',
  'most',
  'much',
  'mucho',
  'neither',
  'none',
  'ones',
  'own',
  'page',
  'part',
  'partly',
  'percent',
  'piec',
  'plenty',
  'portion',
  'remaining',
  'rest',
  'same',
  'section',
  'segment',
  'selection',
  'series',
  'several',
  'significant',
  'simple',
  'singl',
  'some',
  'somewhat',
  'term',
  'ton',
  'tons',
  'total',
  'tripl',
  'unique',
  'various',
  'version',
 

# Dictionary Semantic Embeddings

In [85]:
from empath import Empath
lexicon = Empath()

In [86]:
import spacy
from collections import defaultdict

In [87]:
nlp = spacy.load("en_core_web_lg")

In [88]:
dicts_as_str = defaultdict(str) 
dicts_as_nlp = defaultdict(lambda: 'initial')

In [89]:
for col_key in mass_dict:
    for item in mass_dict[col_key]:
        dicts_as_str[col_key] = dicts_as_str[col_key] + item + " " 
    dicts_as_nlp[col_key] = nlp(dicts_as_str[col_key])

In [90]:
# dicts_as_str = the dictionary as a concatenated string
# dicts_as_nlp = the nlp(objects) of these concatenated string

In [91]:
standard_df_str[0]

0        after almost 15 year of dr not be abl to find ...
1        i read the other comment and wa so surpris . d...
2        with all due respect to the patient who said t...
3        dr steiner is a veri knowledg and care doctor ...
4        i went to him becaus my husband famili all go ...
                               ...                        
71724    dr. stone treat both me and my husband . we li...
71725    such an amaz doctor ... use to be a professor ...
71726    dr. fisher offic staff is veri unfriendli . it...
71727    spend the time with me and doe answer my quest...
71728    excel . knowledg , deliber , respons , level-h...
Name: Review, Length: 71729, dtype: object

In [92]:
dicts_as_nlp.keys()

dict_keys(['Quant', 'Numbers', 'Humans', 'Affect', 'Posemo', 'Negemo', 'Cause', 'Health', 'Money', 'Death', 'Ipron', 'aux_verb', 'adverbs', 'Negate', 'Family', 'Anx', 'Anger', 'Sad', 'CogMech', 'Insight', 'Discrep', 'Tentat', 'Certain', 'Inhib', 'Incl', 'Excl', 'Bio', 'Body', 'Sexual', 'Time', 'Achiev'])

## Semantic Cosine Similarity Embeddings

In [93]:
import statistics as st


In [94]:
def median_cosine_similarity(string_sentence, dict_col, dict_to_compare = dicts_as_nlp):
    ind_dict = dict_to_compare[dict_col]
    main_doc = nlp(string_sentence.lower())
    
    all_scores = set()
    
    for nlp_word in dicts_as_nlp[dict_col]:
        all_scores.add(main_doc.similarity(nlp_word))
    
    return st.median(all_scores)

In [95]:
def create_meta_cols_sem_we(string_sentence, dict_to_compare = dicts_as_nlp):
    # must be a string sentence! not a tokenized one
    # works best with a concatenated string of tokenized words
    new_meta_cols = ['Quant', 
                     'Numbers', 
                     'Humans', 
                     'Affect', 
                     'Posemo', 
                     'Negemo', 
                     'Cause', 
                     'Health', 
                     'Money', 
                     'Death', 
                     'Ipron', 
                     'aux_verb', 
                     'adverbs', 
                     'Negate', 
                     'Family', 
                     'Anx', 
                     'Anger', 
                     'Sad', 
                     'CogMech', 
                     'Insight', 
                     'Discrep', 
                     'Tentat', 
                     'Certain', 
                     'Inhib', 
                     'Incl', 
                     'Excl', 
                     'Bio', 
                     'Body', 
                     'Sexual', 
                     'Time', 
                     'Achiev']
    blank_row_to_fill = pd.DataFrame(columns=new_meta_cols)
    blank_row_to_fill.loc[0] = 0
    
    blank_row_to_fill['Length'] = len(string_sentence.split())
    
    string_doc1 = string_sentence.lower()
    
    for dict_key in dict_to_compare:
        score = median_cosine_similarity(string_doc1, dict_key)        
        blank_row_to_fill[dict_key] = score

    return blank_row_to_fill

In [96]:
def create_meta_dfs_sem_we(train_df_tokenized_strings, test_df_tokenized_strings):
    # must be tokenized string rows
    new_meta_cols = ['Quant', 
                     'Numbers', 
                     'Humans', 
                     'Affect', 
                     'Posemo', 
                     'Negemo', 
                     'Cause', 
                     'Health', 
                     'Money', 
                     'Death', 
                     'Ipron', 
                     'aux_verb', 
                     'adverbs', 
                     'Negate', 
                     'Family', 
                     'Anx', 
                     'Anger', 
                     'Sad', 
                     'CogMech', 
                     'Insight', 
                     'Discrep', 
                     'Tentat', 
                     'Certain', 
                     'Inhib', 
                     'Incl', 
                     'Excl', 
                     'Bio', 
                     'Body', 
                     'Sexual', 
                     'Time', 
                     'Achiev']
    
    meta_train = pd.DataFrame(columns=new_meta_cols)
    meta_test = pd.DataFrame(columns=new_meta_cols)

    for row in train_df_tokenized_strings:
        meta_train = meta_train.append(create_meta_cols_sem_we(row)).reset_index(drop = True)

    for row in test_df_tokenized_strings:
        meta_test = meta_test.append(create_meta_cols_sem_we(row)).reset_index(drop = True)
        
    return meta_train, meta_test

In [ ]:
x_train_meta_data, x_test_meta_data = create_meta_dfs_sem_we(standard_df_str[0], standard_df_str[2])

/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  


In [ ]:
# Grab proper ngram data
x_train_bi_uni = bi_uni_df[0]
y_train_bi_uni = bi_uni_df[1]
x_test_bi_uni = bi_uni_df[2]
y_test_bi_uni = bi_uni_df[3]

In [ ]:
# Grab proper ngram data
x_train_uni = standard_df[0]
y_train_uni = standard_df[1]
x_test_uni = standard_df[2]
y_test_uni = standard_df[3]

In [ ]:
standard_df

In [ ]:
meta_plus_unibi_train = pd.concat([x_train_meta_data, pd.DataFrame(x_train_bi_uni)], axis =1)

In [107]:
meta_plus_unibi_train

,Length,Quant,Numbers,Humans,Affect,Posemo,Negemo,Cause,Health,Money,...,16296,16297,16298,16299,16300,16301,16302,16303,16304,16305
0,12,0.353801,0.337185,0.335885,0.306786,0.323733,0.293186,0.340358,0.216565,0.245024,...,0,0,0,0,0,0,0,0,0,0
1,13,0.445641,0.472066,0.403198,0.318483,0.366868,0.291880,0.389588,0.195020,0.303050,...,0,0,0,0,0,0,0,0,0,0
2,7,0.400896,0.405256,0.334174,0.286815,0.304097,0.271695,0.408599,0.224943,0.270521,...,0,0,0,0,0,0,0,0,0,0
3,12,0.434727,0.427285,0.365813,0.312239,0.347695,0.294671,0.428450,0.233821,0.300104,...,0,0,0,0,0,0,0,0,0,0
4,18,0.380094,0.381828,0.398283,0.322352,0.358448,0.301454,0.365488,0.215303,0.284628,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2,-0.066280,-0.107174,-0.056800,0.009591,-0.005325,0.019552,-0.031881,0.036827,-0.027848,...,0,0,0,0,0,0,0,0,0,0
1996,14,0.439474,0.441441,0.409544,0.326273,0.360026,0.302010,0.413421,0.238387,0.328467,...,0,0,0,0,0,0,0,0,0,0
1997,15,0.395877,0.398751,0.386112,0.309341,0.343397,0.291269,0.350512,0.221610,0.301734,...,0,0,0,0,0,0,0,0,0,0
1998,2,0.167023,0.118744,0.165098,0.197201,0.225408,0.174447,0.142732,0.132263,0.115279,...,0,0,0,0,0,0,0,0,0,0


In [108]:
meta_plus_unibi_test = pd.concat([x_test_meta_data, pd.DataFrame(x_test_bi_uni)], axis =1)

In [109]:
meta_plus_unibi_test

,Length,Quant,Numbers,Humans,Affect,Posemo,Negemo,Cause,Health,Money,...,16296,16297,16298,16299,16300,16301,16302,16303,16304,16305
0,18,0.438788,0.442058,0.376120,0.323262,0.372743,0.295448,0.408184,0.189394,0.323355,...,0,0,0,0,0,0,0,0,0,0
1,9,0.359726,0.311153,0.349979,0.299698,0.322610,0.283746,0.362152,0.185080,0.255581,...,0,0,0,0,0,0,0,0,0,0
2,13,0.381785,0.395566,0.379186,0.310019,0.363533,0.285346,0.347560,0.196813,0.264847,...,0,0,0,0,0,0,0,0,0,0
3,28,0.429036,0.422716,0.399536,0.317259,0.369049,0.291300,0.405328,0.214971,0.320778,...,0,0,0,0,0,0,0,0,0,0
4,32,0.447681,0.463328,0.394265,0.330171,0.363650,0.315782,0.420971,0.227874,0.312311,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,9,0.399572,0.389455,0.370972,0.296135,0.324928,0.280590,0.354530,0.205950,0.282203,...,0,0,0,0,0,0,0,0,0,0
496,9,0.411685,0.391345,0.380572,0.310538,0.340796,0.286594,0.409758,0.211444,0.305251,...,0,0,0,0,0,0,0,0,0,0
497,6,0.390539,0.352376,0.343733,0.288593,0.309180,0.267916,0.381026,0.213367,0.286757,...,0,0,0,0,0,0,0,0,0,0
498,8,0.376050,0.361656,0.398328,0.326424,0.362487,0.306439,0.392490,0.219705,0.291424,...,0,0,0,0,0,0,0,0,0,0


In [110]:
meta_plus_uni_train = pd.concat([x_train_meta_data, pd.DataFrame(x_train_uni)], axis =1)

In [111]:
meta_plus_uni_test = pd.concat([x_test_meta_data, pd.DataFrame(x_test_uni)], axis =1)

# logreg


In [114]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
logreg = LogisticRegressionCV(cv = 5, max_iter = 10000)


In [115]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
df_train_scaled = min_max_scaler.fit_transform(meta_plus_unibi_train)
df_test_scaled = min_max_scaler.fit_transform(meta_plus_unibi_test)


/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [116]:
logreg.fit(df_train_scaled, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=10000,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [117]:
all_eval_metrics(logreg, df_train_scaled, y_train, df_test_scaled, y_test)

Train fit:  0.9815
Test fit:  0.792
Precision:  0.8360655737704918
Recall:  0.5483870967741935
F1:  0.6623376623376623
F0.5:  0.7566765578635014


# Run models on combined set

In [118]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

rf_model = RandomForestClassifier(criterion = 'entropy') 

In [119]:
# 5-Fold Cross validation
np.mean(cross_val_score(rf_model, meta_plus_unibi_train, y_train, cv=5))

/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/juliabarn

0.7870376721104506

In [122]:
intro_param_grid = {
                    'bootstrap': [True],
                    'criterion': ['entropy'],
                    'max_depth': [20, 25],
                    'max_features': ['sqrt'],
                    'min_samples_leaf': [1, 2, 3],
                    'min_samples_split': [3, 5],
                    'n_estimators': [1000]
                   }

In [123]:
from sklearn.model_selection import GridSearchCV

grid_rf_model = GridSearchCV(rf_model, intro_param_grid, cv=5)
grid_rf_model.fit(meta_plus_unibi_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'bootstrap': [True], 'criterion': ['entropy'], 'max_depth': [20, 25], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 2, 3], 'min_samples_split': [3, 5], 'n_estimators': [1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [124]:
meta_plus_unibi_train

,Length,Quant,Numbers,Humans,Affect,Posemo,Negemo,Cause,Health,Money,...,16296,16297,16298,16299,16300,16301,16302,16303,16304,16305
0,12,0.353801,0.337185,0.335885,0.306786,0.323733,0.293186,0.340358,0.216565,0.245024,...,0,0,0,0,0,0,0,0,0,0
1,13,0.445641,0.472066,0.403198,0.318483,0.366868,0.291880,0.389588,0.195020,0.303050,...,0,0,0,0,0,0,0,0,0,0
2,7,0.400896,0.405256,0.334174,0.286815,0.304097,0.271695,0.408599,0.224943,0.270521,...,0,0,0,0,0,0,0,0,0,0
3,12,0.434727,0.427285,0.365813,0.312239,0.347695,0.294671,0.428450,0.233821,0.300104,...,0,0,0,0,0,0,0,0,0,0
4,18,0.380094,0.381828,0.398283,0.322352,0.358448,0.301454,0.365488,0.215303,0.284628,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2,-0.066280,-0.107174,-0.056800,0.009591,-0.005325,0.019552,-0.031881,0.036827,-0.027848,...,0,0,0,0,0,0,0,0,0,0
1996,14,0.439474,0.441441,0.409544,0.326273,0.360026,0.302010,0.413421,0.238387,0.328467,...,0,0,0,0,0,0,0,0,0,0
1997,15,0.395877,0.398751,0.386112,0.309341,0.343397,0.291269,0.350512,0.221610,0.301734,...,0,0,0,0,0,0,0,0,0,0
1998,2,0.167023,0.118744,0.165098,0.197201,0.225408,0.174447,0.142732,0.132263,0.115279,...,0,0,0,0,0,0,0,0,0,0


In [151]:
# grid_rf_model.best_estimator_ 

In [152]:
# grid_rf_model.best_params_

In [153]:
all_eval_metrics(grid_rf_model, meta_plus_unibi_train, y_train, meta_plus_unibi_test, y_test)

Train fit:  0.8075
Test fit:  0.756
Precision:  0.7622950819672131
Recall:  0.5
F1:  0.6038961038961039
F0.5:  0.6899109792284867


# CHANGE METRIC

In [127]:
from sklearn.metrics import fbeta_score, make_scorer

In [128]:
fone_scorer = make_scorer(fbeta_score, beta=0.5)
fone_scorer

make_scorer(fbeta_score, beta=0.5)

In [129]:
rf_model_f1 = RandomForestClassifier(criterion = 'entropy') 

In [130]:
param_grid1 = {
                 'n_estimators': [1000, 1100],
                 'max_depth': [15, 20, 25],
                 'criterion': ["entropy"],
                 'max_features': ['auto', 'sqrt'],
                 'min_samples_split': [2, 3, 4],
                 'min_samples_leaf': [1, 2],
                 'bootstrap': [True, False]
             }

In [131]:
from sklearn.model_selection import GridSearchCV

grid_rf_model_f1 = GridSearchCV(rf_model_f1, param_grid1, cv=5, scoring=fone_scorer)
grid_rf_model_f1.fit(meta_plus_uni_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1000, 1100], 'max_depth': [15, 20, 25], 'criterion': ['entropy'], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(fbeta_score, beta=0.5), verbose=0)

In [132]:
grid_rf_model_f1.best_estimator_ 

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=25, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [133]:
grid_rf_model_f1.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 25,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [134]:
all_eval_metrics(grid_rf_model_f1, meta_plus_uni_train, y_train, meta_plus_uni_test, y_test)

Train fit:  0.9976507439310885
Test fit:  0.8108866442199775
Precision:  0.8120300751879699
Recall:  0.5806451612903226
F1:  0.677115987460815
F0.5:  0.7520891364902506


In [135]:
preds = grid_rf_model_f1.predict(meta_plus_uni_test)

In [136]:
grid_rf_model_f1.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 25,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [137]:
best_params = {'bootstrap': [False],
 'criterion': ['entropy'],
 'max_depth': [25],
 'max_features': ['sqrt'],
 'min_samples_leaf': [1],
 'min_samples_split': [2],
 'n_estimators': [1000]}

In [138]:
rf_model_best = RandomForestClassifier(criterion = 'entropy') 

In [142]:
from sklearn.model_selection import GridSearchCV

grid_rf_model_best = GridSearchCV(rf_model_best, best_params, cv=5, scoring=fone_scorer)
grid_rf_model_best.fit(meta_plus_uni_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'bootstrap': [False], 'criterion': ['entropy'], 'max_depth': [25], 'max_features': ['sqrt'], 'min_samples_leaf': [1], 'min_samples_split': [2], 'n_estimators': [1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(fbeta_score, beta=0.5), verbose=0)

In [143]:
meta_plus_uni_train

,Length,Quant,Numbers,Humans,Affect,Posemo,Negemo,Cause,Health,Money,...,2230,2231,2232,2233,2234,2235,2236,2237,2238,2239
0,12,0.353801,0.337185,0.335885,0.306786,0.323733,0.293186,0.340358,0.216565,0.245024,...,0,0,0,0,0,0,0,0,0,0
1,13,0.445641,0.472066,0.403198,0.318483,0.366868,0.291880,0.389588,0.195020,0.303050,...,0,0,0,0,0,0,0,0,0,0
2,7,0.400896,0.405256,0.334174,0.286815,0.304097,0.271695,0.408599,0.224943,0.270521,...,0,0,0,0,0,0,0,0,0,0
3,12,0.434727,0.427285,0.365813,0.312239,0.347695,0.294671,0.428450,0.233821,0.300104,...,0,0,0,0,0,0,0,0,0,0
4,18,0.380094,0.381828,0.398283,0.322352,0.358448,0.301454,0.365488,0.215303,0.284628,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2,-0.066280,-0.107174,-0.056800,0.009591,-0.005325,0.019552,-0.031881,0.036827,-0.027848,...,0,0,0,0,0,0,0,0,0,0
1996,14,0.439474,0.441441,0.409544,0.326273,0.360026,0.302010,0.413421,0.238387,0.328467,...,0,0,0,0,0,0,0,0,0,0
1997,15,0.395877,0.398751,0.386112,0.309341,0.343397,0.291269,0.350512,0.221610,0.301734,...,0,0,0,0,0,0,0,0,0,0
1998,2,0.167023,0.118744,0.165098,0.197201,0.225408,0.174447,0.142732,0.132263,0.115279,...,0,0,0,0,0,0,0,0,0,0


In [144]:
all_eval_metrics(grid_rf_model_best, meta_plus_uni_train, y_train, meta_plus_uni_test, y_test)

Train fit:  0.9976507439310885
Test fit:  0.8044692737430168
Precision:  0.8015267175572519
Recall:  0.5645161290322581
F1:  0.662460567823344
F0.5:  0.7394366197183099


### Bigram Inclusion

In [148]:
rf_model_f1_bi = RandomForestClassifier(criterion = 'entropy') 

In [150]:
param_grid1 = {
                 'n_estimators': [1000, 1100],
                 'max_depth': [15, 20, 25],
                 'criterion': ["entropy"],
                 'max_features': ['auto', 'sqrt'],
                 'min_samples_split': [2, 3, 4],
                 'min_samples_leaf': [1, 2],
                 'bootstrap': [True, False]
             }

In [151]:
from sklearn.model_selection import GridSearchCV

grid_rf_model_f1_bi = GridSearchCV(rf_model_f1_bi, param_grid1, cv=5, scoring=fone_scorer)
grid_rf_model_f1_bi.fit(meta_plus_unibi_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1000, 1100], 'max_depth': [15, 20, 25], 'criterion': ['entropy'], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(fbeta_score, beta=0.5), verbose=0)

In [152]:
grid_rf_model_f1_bi.best_estimator_ 

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=25, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [153]:
grid_rf_model_f1_bi.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 25,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [154]:
all_eval_metrics(grid_rf_model_f1_bi, meta_plus_unibi_train, y_train, meta_plus_unibi_test, y_test)

Train fit:  0.915523465703971
Test fit:  0.7860962566844921
Precision:  0.8198198198198198
Recall:  0.489247311827957
F1:  0.6127946127946127
F0.5:  0.7222222222222222


In [155]:
#preds = grid_rf_model_f1_bi.predict(meta_plus_unibi_test)

In [137]:
best_params_bi = {'bootstrap': [False],
 'criterion': ['entropy'],
 'max_depth': [25],
 'max_features': ['sqrt'],
 'min_samples_leaf': [1],
 'min_samples_split': [2],
 'n_estimators': [1000]}

In [138]:
rf_model_best_bi = RandomForestClassifier(criterion = 'entropy') 

In [142]:
from sklearn.model_selection import GridSearchCV

grid_rf_model_best_bi = GridSearchCV(rf_model_best_bi, best_params_bi, cv=5, scoring=fone_scorer)
grid_rf_model_best_bi.fit(meta_plus_unibi_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'bootstrap': [False], 'criterion': ['entropy'], 'max_depth': [25], 'max_features': ['sqrt'], 'min_samples_leaf': [1], 'min_samples_split': [2], 'n_estimators': [1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(fbeta_score, beta=0.5), verbose=0)

In [144]:
all_eval_metrics(grid_rf_model_best, meta_plus_unibi_train, y_train, meta_plus_unibi_test, y_test)



Train fit:  0.9976507439310885
Test fit:  0.8044692737430168
Precision:  0.8015267175572519
Recall:  0.5645161290322581
F1:  0.662460567823344
F0.5:  0.7394366197183099


# Logisitic Regression

In [181]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
logreg = LogisticRegressionCV(cv = 5, max_iter = 10000)


In [189]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
df_train_scaled = min_max_scaler.fit_transform(meta_plus_unibi_train)
df_test_scaled = min_max_scaler.fit_transform(meta_plus_unibi_test)


/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/Users/juliabarnett/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [210]:
param_grid_logr = {    
                     'fit_intercept': [True, False],
                     'max_iter': [50000],
                     'penalty': ['l2'],
                     'refit': [True, False                     
                     'scoring': [fone_scorer],
                     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
}


In [211]:
grid_rf_model_regr = GridSearchCV(logreg, param_grid_logr, cv=5, scoring=fone_scorer)
grid_rf_model_regr.fit(df_train_scaled, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=10000,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [True, False], 'max_iter': [50000], 'penalty': ['l2'], 'refit': [True, False], 'scoring': [make_scorer(fbeta_score, beta=0.5)], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(fbeta_score, beta=0.5), verbose=0)

In [212]:
all_eval_metrics(grid_rf_model_regr, df_train_scaled, y_train, df_test_scaled, y_test)

Train fit:  0.9954609485052435
Test fit:  0.8333333333333334
Precision:  0.8380281690140845
Recall:  0.6397849462365591
F1:  0.725609756097561
F0.5:  0.7891246684350132


In [223]:
def conf_matrix(pred_val, actual_val):
    if pred_val == 1:
        if actual_val == 1:
            return "TP"
        elif actual_val == 0:
            return "FP"
    elif pred_val == 0:
        if actual_val == 1:
            return "FN"
        elif actual_val == 0:
            return "TN"
    else:
        return "ERROR IN DATA"

In [224]:
def create_conf_df(model, x_df, y_df):
    compared_df = pd.DataFrame()
    compared_df["preds"] = model.predict(x_df)
    compared_df["actuals"] = y_df
    compared_df["error_type"] = compared_df.apply(lambda x: conf_matrix(x.preds, x.actuals), axis=1)
    compared_df["count"] = 1
    return compared_df

In [225]:
def summary_cost_matrix(model, x_df, y_df):
    conf_df = create_conf_df(model, x_df, y_df)
    summary_df = conf_df[["error_type", "count"]].groupby(by=["error_type"]).sum()
    fn = summary_df.iloc[0][0]
    fp = summary_df.iloc[1][0]
    tn = summary_df.iloc[2][0]
    tp = summary_df.iloc[3][0]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = (2*precision*recall)/(precision+recall)
    F0_5score = ((1 + 0.5**2) * precision * recall) / (0.5**2 * precision + recall)
    print(summary_df)
    print("FN:", fn)
    print("FP:", fp)
    print("TN:", tn)
    print("TP:", tp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1_score)
    print("F0.5:", F0_5score)

In [226]:
def fitness_check(model, x_train, y_train, x_test, y_test):
    train_fit = model.score(x_train, y_train)
    print("Train fit: ", train_fit)
    test_fit = model.score(x_test, y_test)
    print("Test fit: ", test_fit)

In [227]:
fitness_check(grid_rf_model_regr, df_train_scaled, y_train, df_test_scaled, y_test)

Train fit:  0.9954609485052435
Test fit:  0.8333333333333334


In [228]:
summary_cost_matrix(grid_rf_model_regr, df_test_scaled, y_test)

            count
error_type       
FN             23
FP             67
TN            119
TP            291
FN: 23
FP: 67
TN: 119
TP: 291
Precision: 0.8128491620111732
Recall: 0.9267515923566879
F1: 0.8660714285714286
F0.5: 0.8333333333333334
